In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "data/투자설명서.pdf"
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = loader.load_and_split(doc_splitter)

In [2]:
from langchain_community.retrievers import BM25Retriever
from kiwipiepy import Kiwi

kiwi_tokenizer = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi_tokenizer.tokenize(text)]

- Kiwi라는 한국어 형태소 분석기를 kiwipiepy 라이브러리에서 임포트 합니다.
- 입력된 텍스트를 Kiwi를 이용해 토큰화하고, 각 토큰의 기본형(form)만을 리스트로 반환합니다. => 형태소 단위로 분리

In [3]:
bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)
bm25_retriever.k = 2

- 각 문서를 인덱싱할 때 kiwi_tokenize 함수를 사용하여 전처리하라는 의미입니다.
- k=2로 검색시 쿼리와 가장 관련성 높은 상위 2개의 문서만을 반환하도록 지정하는 옵션입니다.
- 이를 바탕으로 최종 답변을 생성하는 단계로 넘어갑니다. RetrievalQA체인을 활용합니다.

In [5]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

# 관련 있는 문서 수집 후, 챗GPT로 최종 답변까지 수행하는 체인을 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.2),
    chain_type="stuff",
    retriever=bm25_retriever,
    return_source_documents=True, # 답변에 사용된 source document도 보여주도록 설정
)

- RetrievalQA와 ConversationRetrievalChain은 검색과 질문-답변 기능을 제공하는 체인 클래스입니다.
- 파라미터
  - llm: ChatOpenAI 인스턴스를 생성하여 사용합니다. temperature를 0.2로 설정하여 비교적 일관된 출력을 얻도록 합니다.
  - chain_type: stuff로 설정합니다. 이는 모든 관련 문서를 하나의 컨텍스트로 결합하여 LLM에 전달하는 방식입니다.
  - retriever: 이전에 생성한 bm25_retriever를 사용하여 질문과 관련된 문서를 검색합니다.
  - return_source_documents: True로 설정하여 답변에 사용된 원본 문서의 정보도 함께 반환하도록 합니다.

In [6]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량이 어느 정도야?")

{'query': '이 회사가 발행한 주식의 총 발행량이 어느 정도야?',
 'result': '이 회사가 발행한 주식의 총 발행량은 13,602,977주입니다.',
 'source_documents': [Document(metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': 'data/투자설명서.pdf', 'total_pages': 514, 'page': 341, 'page_label': '342'}, page_content='당사는 투기적 목적으로 파생금융상품을 포함한 금융상품계약을 체결하거나 거래하지 않습\n니다. \n  \n바. 파생거래 \n  \n당사가 2021년 3월 19일에 발행한 제2회 무기명식 무보증 사모 전환사채에는 조기상환청구\n권(Put option)과 중도상환청구권(Call option)이 포함되어 있습니다. 당분기말 현재 전환사\n채의 내용은 다음과 같습니다. \n구 \xa0분 내역\n사채의 종류 제2회 무기명식 무보증 사모 전환사채\n발행가액 (단위: 천원) 19,000,000\n미상환사채 권면총액 (단위: 천원\n)(*1) 7,600,000\n이자지급조건 없음\n연이자율 0.0%\n보장수익률 0.0%\n발행일 2021년 3월 19일\n상환일 2026년 3월 19일\n전환청구기간\n시작일 2022년 3월 19일\n종료일 2026년 2월 19일\n전환에 따라\n발행할 주식\n종류\n주식회사 셀리드 기명식 보통주\n식\n전환가액(원/주)(*2) 4,515\n주식수(*2) 1,683,277주\n전환비율(%) 100\n전환가액의 조정조건\n1.본건 전환사채의 전환가액을 하회하는 발행가액, 전환가액 또\n는 행사가액으로 유상증자 또는 그에 준하는 주식연계증